In [1]:
import pandas as pd

Factors that may determine property value:

External - Income growth, unemployment, population growth, interest rates
Intrinsic - Dist to MRT, HAWKER, SCHOOLS, CBD
            Size
            Age of building
            View/Orientation
            Height
            Demographics
            

Data Sources: 
Income  - https://data.gov.sg/dataset/income-of-individuals-by-income-type-annual 

Unemployment - https://data.gov.sg/dataset/jvetoue-qtrly

Distance to amenities - https://data.gov.sg/dataset/resale-flat-prices + HAWKERS https://data.gov.sg/dataset/list-of-government-markets-hawker-centres + MRT https://github.com/hxchua/datadoubleconfirm/blob/master/datasets/mrtsg.csv + SCHOOLS (pri to pre-tieriary) https://data.gov.sg/dataset/school-directory-and-information?resource_id=ede26d32-01af-4228-b1ed-f05c45a1d8ee + Using Cityhall mrt as CBD

Demographics - https://www.singstat.gov.sg/find-data/search-by-theme/population/geographic-distribution/latest-data

Value of property - https://data.gov.sg/dataset/resale-flat-prices

Height - https://data.gov.sg/dataset/hdb-property-information

Interest Rates - https://eservices.mas.gov.sg/statistics/dir/DomesticInterestRates.aspx


Questions to answer:

What factors influence property prices?




In [2]:
value_df = pd.read_csv("Resale value 2017-now.csv")
value_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
122671,2022-03,YISHUN,EXECUTIVE,355,YISHUN RING RD,10 TO 12,146.0,Maisonette,1988,65 years 08 months,800000.0
122672,2022-03,YISHUN,EXECUTIVE,277,YISHUN ST 22,10 TO 12,146.0,Maisonette,1985,62 years 06 months,788000.0
122673,2022-03,YISHUN,EXECUTIVE,824,YISHUN ST 81,01 TO 03,142.0,Apartment,1987,64 years 09 months,800000.0
122674,2022-03,YISHUN,EXECUTIVE,828,YISHUN ST 81,01 TO 03,142.0,Apartment,1988,64 years 11 months,723000.0


In [3]:
value_df['address'] = value_df['block'] + ' ' + value_df['street_name']
all_address = list(value_df['address'])
unique_address = list(set(all_address))
print('Number of unique addresses: ' + str(len(unique_address)))

Number of unique addresses: 9209


In [4]:
unique_address

['251 COMPASSVALE ST',
 '534 BT BATOK ST 51',
 '110C PUNGGOL FIELD',
 '406 PANDAN GDNS',
 '742 YISHUN AVE 5',
 '203 CHOA CHU KANG AVE 1',
 '424 PASIR RIS DR 6',
 '651 PASIR RIS DR 10',
 '104 WOODLANDS ST 13',
 '437 JURONG WEST AVE 1',
 '926 HOUGANG ST 91',
 '359 YUNG AN RD',
 '612A PUNGGOL DR',
 '701 YISHUN AVE 5',
 '304 HOUGANG AVE 5',
 '3 BEDOK STH AVE 1',
 '530A PASIR RIS DR 1',
 '230 YISHUN ST 21',
 '335 UBI AVE 1',
 '506 BT BATOK ST 52',
 '764 YISHUN ST 72',
 '671B KLANG LANE',
 '615 ELIAS RD',
 '518 PASIR RIS ST 52',
 '702 YISHUN AVE 5',
 '405 ADMIRALTY LINK',
 '489B CHOA CHU KANG AVE 5',
 '231 PASIR RIS DR 4',
 '321 SERANGOON AVE 2',
 '452 JURONG WEST ST 42',
 '10A BOON TIONG RD',
 '635 VEERASAMY RD',
 '149 PASIR RIS ST 13',
 '46 JLN BAHAGIA',
 '187 BISHAN ST 13',
 '172 ANG MO KIO AVE 4',
 '182 JELEBU RD',
 '1 CHANGI VILLAGE RD',
 '216 BISHAN ST 23',
 '106 BEDOK RESERVOIR RD',
 '351 HOUGANG AVE 7',
 '43 TELOK BLANGAH RISE',
 '32 MARSILING DR',
 '121D SENGKANG EAST WAY',
 '616 SE

In [5]:
#Begin merging with data on distance
#Use property locations in value_df to obtain coordinates

import requests
import json

def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        
        
        # Retrieve information from website
        response = requests.get(url)
        data = json.loads(response.text) 
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')



# add = "667C JURONG WEST ST 65"
# url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+add
# response = requests.get(url)
# data = json.loads(response.text) 
# data



In [6]:
hdblocation_df = pd.read_csv('hdb_coordinates.csv')
hdblocation_df

,Unnamed: 0,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,0,215 LORONG 8 TOA PAYOH SINGAPORE 310215,215,LORONG 8 TOA PAYOH,NIL,215 LORONG 8 TOA PAYOH SINGAPORE 310215,310215,30424.918340,35998.064931,1.341828,103.855108,103.855108,215 LOR 8 TOA PAYOH
1,0,TOA PAYOH VIEW,118,LORONG 1 TOA PAYOH,TOA PAYOH VIEW,118 LORONG 1 TOA PAYOH TOA PAYOH VIEW SINGAPOR...,310118,29288.175336,35747.591826,1.339563,103.844894,103.844894,118 LOR 1 TOA PAYOH
2,0,699 HOUGANG STREET 52 SINGAPORE 530699,699,HOUGANG STREET 52,NIL,699 HOUGANG STREET 52 SINGAPORE 530699,530699,34217.826348,39767.903832,1.375921,103.889190,103.889190,699 HOUGANG ST 52
3,0,483 CHOA CHU KANG AVENUE 5 SINGAPORE 680483,483,CHOA CHU KANG AVENUE 5,NIL,483 CHOA CHU KANG AVENUE 5 SINGAPORE 680483,680483,17365.222944,39759.289957,1.375841,103.737757,103.737757,483 CHOA CHU KANG AVE 5
4,1,GREENERY CHILDCARE CENTRE,483,CHOA CHU KANG AVENUE 5,GREENERY CHILDCARE CENTRE,483 CHOA CHU KANG AVENUE 5 GREENERY CHILDCARE ...,680483,17365.223096,39759.289897,1.375841,103.737757,103.737757,483 CHOA CHU KANG AVE 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11383,0,COMPASSVALE VISTA,224A,COMPASSVALE WALK,COMPASSVALE VISTA,224A COMPASSVALE WALK COMPASSVALE VISTA SINGAP...,541224,35200.334816,41090.587237,1.387882,103.898019,103.898019,224A COMPASSVALE WALK
11384,0,212 JURONG EAST STREET 21 SINGAPORE 600212,212,JURONG EAST STREET 21,NIL,212 JURONG EAST STREET 21 SINGAPORE 600212,600212,17658.260007,35695.924923,1.339094,103.740392,103.740392,212 JURONG EAST ST 21
11385,0,681C WOODLANDS DRIVE 62 SINGAPORE 733681,681C,WOODLANDS DRIVE 62,NIL,681C WOODLANDS DRIVE 62 SINGAPORE 733681,733681,24639.376585,46830.054492,1.439789,103.803120,103.803120,681C WOODLANDS DR 62
11386,0,FERN SPRING,403B,FERNVALE LANE,FERN SPRING,403B FERNVALE LANE FERN SPRING SINGAPORE 792403,792403,32413.782632,41191.053425,1.388791,103.872980,103.872980,403B FERNVALE LANE


In [7]:
hdblocation_df.rename( columns={'Unnamed: 0':'Count'}, inplace=True )
hdblocation_df

,Count,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,0,215 LORONG 8 TOA PAYOH SINGAPORE 310215,215,LORONG 8 TOA PAYOH,NIL,215 LORONG 8 TOA PAYOH SINGAPORE 310215,310215,30424.918340,35998.064931,1.341828,103.855108,103.855108,215 LOR 8 TOA PAYOH
1,0,TOA PAYOH VIEW,118,LORONG 1 TOA PAYOH,TOA PAYOH VIEW,118 LORONG 1 TOA PAYOH TOA PAYOH VIEW SINGAPOR...,310118,29288.175336,35747.591826,1.339563,103.844894,103.844894,118 LOR 1 TOA PAYOH
2,0,699 HOUGANG STREET 52 SINGAPORE 530699,699,HOUGANG STREET 52,NIL,699 HOUGANG STREET 52 SINGAPORE 530699,530699,34217.826348,39767.903832,1.375921,103.889190,103.889190,699 HOUGANG ST 52
3,0,483 CHOA CHU KANG AVENUE 5 SINGAPORE 680483,483,CHOA CHU KANG AVENUE 5,NIL,483 CHOA CHU KANG AVENUE 5 SINGAPORE 680483,680483,17365.222944,39759.289957,1.375841,103.737757,103.737757,483 CHOA CHU KANG AVE 5
4,1,GREENERY CHILDCARE CENTRE,483,CHOA CHU KANG AVENUE 5,GREENERY CHILDCARE CENTRE,483 CHOA CHU KANG AVENUE 5 GREENERY CHILDCARE ...,680483,17365.223096,39759.289897,1.375841,103.737757,103.737757,483 CHOA CHU KANG AVE 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11383,0,COMPASSVALE VISTA,224A,COMPASSVALE WALK,COMPASSVALE VISTA,224A COMPASSVALE WALK COMPASSVALE VISTA SINGAP...,541224,35200.334816,41090.587237,1.387882,103.898019,103.898019,224A COMPASSVALE WALK
11384,0,212 JURONG EAST STREET 21 SINGAPORE 600212,212,JURONG EAST STREET 21,NIL,212 JURONG EAST STREET 21 SINGAPORE 600212,600212,17658.260007,35695.924923,1.339094,103.740392,103.740392,212 JURONG EAST ST 21
11385,0,681C WOODLANDS DRIVE 62 SINGAPORE 733681,681C,WOODLANDS DRIVE 62,NIL,681C WOODLANDS DRIVE 62 SINGAPORE 733681,733681,24639.376585,46830.054492,1.439789,103.803120,103.803120,681C WOODLANDS DR 62
11386,0,FERN SPRING,403B,FERNVALE LANE,FERN SPRING,403B FERNVALE LANE FERN SPRING SINGAPORE 792403,792403,32413.782632,41191.053425,1.388791,103.872980,103.872980,403B FERNVALE LANE


In [8]:
hdblocation_df = hdblocation_df.loc[hdblocation_df['Count'] == 0]
hdblocation_df


,Count,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,0,215 LORONG 8 TOA PAYOH SINGAPORE 310215,215,LORONG 8 TOA PAYOH,NIL,215 LORONG 8 TOA PAYOH SINGAPORE 310215,310215,30424.918340,35998.064931,1.341828,103.855108,103.855108,215 LOR 8 TOA PAYOH
1,0,TOA PAYOH VIEW,118,LORONG 1 TOA PAYOH,TOA PAYOH VIEW,118 LORONG 1 TOA PAYOH TOA PAYOH VIEW SINGAPOR...,310118,29288.175336,35747.591826,1.339563,103.844894,103.844894,118 LOR 1 TOA PAYOH
2,0,699 HOUGANG STREET 52 SINGAPORE 530699,699,HOUGANG STREET 52,NIL,699 HOUGANG STREET 52 SINGAPORE 530699,530699,34217.826348,39767.903832,1.375921,103.889190,103.889190,699 HOUGANG ST 52
3,0,483 CHOA CHU KANG AVENUE 5 SINGAPORE 680483,483,CHOA CHU KANG AVENUE 5,NIL,483 CHOA CHU KANG AVENUE 5 SINGAPORE 680483,680483,17365.222944,39759.289957,1.375841,103.737757,103.737757,483 CHOA CHU KANG AVE 5
5,0,SEMBAWANG GREEN,486,ADMIRALTY LINK,SEMBAWANG GREEN,486 ADMIRALTY LINK SEMBAWANG GREEN SINGAPORE 7...,750486,26083.865258,48625.638185,1.456027,103.816100,103.816100,486 ADMIRALTY LINK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11383,0,COMPASSVALE VISTA,224A,COMPASSVALE WALK,COMPASSVALE VISTA,224A COMPASSVALE WALK COMPASSVALE VISTA SINGAP...,541224,35200.334816,41090.587237,1.387882,103.898019,103.898019,224A COMPASSVALE WALK
11384,0,212 JURONG EAST STREET 21 SINGAPORE 600212,212,JURONG EAST STREET 21,NIL,212 JURONG EAST STREET 21 SINGAPORE 600212,600212,17658.260007,35695.924923,1.339094,103.740392,103.740392,212 JURONG EAST ST 21
11385,0,681C WOODLANDS DRIVE 62 SINGAPORE 733681,681C,WOODLANDS DRIVE 62,NIL,681C WOODLANDS DRIVE 62 SINGAPORE 733681,733681,24639.376585,46830.054492,1.439789,103.803120,103.803120,681C WOODLANDS DR 62
11386,0,FERN SPRING,403B,FERNVALE LANE,FERN SPRING,403B FERNVALE LANE FERN SPRING SINGAPORE 792403,792403,32413.782632,41191.053425,1.388791,103.872980,103.872980,403B FERNVALE LANE


In [9]:
hdblocation_df = hdblocation_df[['LATITUDE','LONGITUDE','address']]
hdblocation_df

,LATITUDE,LONGITUDE,address
0,1.341828,103.855108,215 LOR 8 TOA PAYOH
1,1.339563,103.844894,118 LOR 1 TOA PAYOH
2,1.375921,103.889190,699 HOUGANG ST 52
3,1.375841,103.737757,483 CHOA CHU KANG AVE 5
5,1.456027,103.816100,486 ADMIRALTY LINK
...,...,...,...
11383,1.387882,103.898019,224A COMPASSVALE WALK
11384,1.339094,103.740392,212 JURONG EAST ST 21
11385,1.439789,103.803120,681C WOODLANDS DR 62
11386,1.388791,103.872980,403B FERNVALE LANE


In [10]:
hawker_df = pd.read_csv('Hawker Location.csv')
hawker_df = hawker_df['location_of_centre']

In [11]:
hawker_address = list(set(hawker_df))

In [12]:
hawker_address

['Blk 89, Circuit Road, S(370089)',
 'Blk 137, Tampines Street 11, S(521137)',
 'Blk 6, Jalan Bukit Merah, S(150006)',
 'Blk 13/14, Haig Road, S(430013/430014)',
 'Blk 163, Bukit Merah Central, S(150163)',
 'Blk 120, Bukit Merah Lane, S(150120)',
 'Blk 270, Queen Street, S(180270)',
 'Blk 85, Redhill Lane, S(150085)',
 'Blk 32, New Market Road, S(050032)',
 '20, Kensington Park Road, S(557269)',
 '3, Yung Sheng Road, S(618499)',
 'Blk 69, Geylang Bahru, S(330069)',
 'Blk 254, Jurong East Street 24, S(600254)',
 'Blk 538, Bedok North Street 3, S(460538)',
 '166, Jalan Besar, S(208877)',
 'Blk 58, New Upper Changi Road, S(461058)',
 'Blk 726, Clementi West Street 2, S(120726)',
 'Blk 347, Jurong East Ave 1, S(600347)',
 'Blk 29, Bendemeer Road, S(330029)',
 'Blk 208B, New Upper Changi Road, S(462208)',
 'Blk 502, West Coast Drive, S(120502)',
 'Blk 1, Jalan Kukoh, S(161001)',
 'Blk 105, Hougang Ave 1, S(530105)',
 'Blk 505, Jurong West Street 52, S(640505)',
 'National Development Buildi

In [13]:
hawker_address = [x[:-1] for x in hawker_address]
hawker_address = [x[-6:] for x in hawker_address]

In [14]:
hawker_address

['370089',
 '521137',
 '150006',
 '430014',
 '150163',
 '150120',
 '180270',
 '150085',
 '050032',
 '557269',
 '618499',
 '330069',
 '600254',
 '460538',
 '208877',
 '461058',
 '120726',
 '600347',
 '330029',
 '462208',
 '120502',
 '161001',
 '530105',
 '640505',
 '069111',
 '560527',
 '530209',
 '310210',
 '560724',
 '568226',
 '210665',
 '270020',
 '118543',
 '150079',
 '370080',
 '440084',
 '069184',
 '168898',
 '270044',
 '432004',
 '320092',
 '081006',
 '120448',
 '247792',
 '310127',
 '149644',
 '642221',
 '564226',
 '424768',
 '441050',
 '310093',
 '310022',
 '588215',
 '570320',
 '561453',
 '151115',
 '760105',
 '402004',
 '555945',
 '560162',
 '260007',
 '397972',
 '601037',
 '090036',
 '380049',
 '574419',
 '211041',
 '162022',
 '460216',
 '199583',
 '100082',
 '320090',
 '402001',
 '120503',
 '380117',
 '470630',
 '560628',
 '468960',
 '050335',
 '289876',
 '460016',
 '460085',
 '371079',
 '310075',
 '469572',
 '500003',
 '160112',
 '731004',
 '051531',
 '310074',
 '100079',

In [15]:
len(hawker_address)

107

In [16]:
# find_postal(hawker_address, 'hawker_coordinates')

In [17]:
hawkerlocation_df = pd.read_csv('hawker_coordinates.csv')
hawkerlocation_df = hawkerlocation_df.drop(columns=['ADDRESS'])

In [18]:
hawkerlocation_df

,Unnamed: 0,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,0,MARKET & HAWKER CENTRE (BLKS 20/21 MARSILING L...,21,MARSILING LANE,MARKET & HAWKER CENTRE (BLKS 20/21 MARSILING L...,730021,21756.033111,47277.163037,1.443832,103.777210,103.777210,730021
1,1,MARSILING LANE MARKET & COOKED FOOD CENTRE,21,MARSILING LANE,MARSILING LANE MARKET & COOKED FOOD CENTRE,730021,21755.097618,47287.482967,1.443925,103.777202,103.777202,730021
2,0,DBS TELOK BLANGAH RISE,36,TELOK BLANGAH RISE,DBS TELOK BLANGAH RISE,90036,26743.489737,28343.943518,1.272607,103.822028,103.822028,90036
3,1,TELOK BLANGAH RISE MARKET,36,TELOK BLANGAH RISE,TELOK BLANGAH RISE MARKET,90036,26780.789700,28360.999769,1.272761,103.822364,103.822364,90036
4,0,DBS CLEMENTI WEST STREET 2,726,CLEMENTI WEST STREET 2,DBS CLEMENTI WEST STREET 2,120726,20314.909390,31791.216699,1.303782,103.764264,103.764264,120726
...,...,...,...,...,...,...,...,...,...,...,...,...
168,0,MARKET & HAWKER CENTRE (BLK 448 CLEMENTI AVENU...,448,CLEMENTI AVENUE 3,MARKET & HAWKER CENTRE (BLK 448 CLEMENTI AVENU...,120448,20343.061339,32845.842058,1.313320,103.764517,103.764517,120448
169,0,MARKET & HAWKER CENTRE (BLK 216 BEDOK NORTH ST...,216,BEDOK NORTH STREET 1,MARKET & HAWKER CENTRE (BLK 216 BEDOK NORTH ST...,460216,39117.792247,34365.596824,1.327063,103.933218,103.933218,460216
170,0,DBS EAST COAST LAGOON FOOD VILLAGE,1220,EAST COAST PARKWAY,DBS EAST COAST LAGOON FOOD VILLAGE,468960,39331.556190,32136.743773,1.306906,103.935138,103.935138,468960
171,1,EAST COAST LAGOON FOOD VILLAGE,1220,EAST COAST PARKWAY,EAST COAST LAGOON FOOD VILLAGE,468960,39293.916166,32176.032917,1.307261,103.934800,103.934800,468960


In [19]:
hawkerlocation_df = hawkerlocation_df.loc[hawkerlocation_df['Unnamed: 0'] == 0]
hawkerlocation_df.dtypes


Unnamed: 0      int64
SEARCHVAL      object
BLK_NO         object
ROAD_NAME      object
BUILDING       object
POSTAL          int64
X             float64
Y             float64
LATITUDE      float64
LONGITUDE     float64
LONGTITUDE    float64
address         int64
dtype: object

In [20]:
hawkerlocation_df

,Unnamed: 0,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,0,MARKET & HAWKER CENTRE (BLKS 20/21 MARSILING L...,21,MARSILING LANE,MARKET & HAWKER CENTRE (BLKS 20/21 MARSILING L...,730021,21756.033111,47277.163037,1.443832,103.777210,103.777210,730021
2,0,DBS TELOK BLANGAH RISE,36,TELOK BLANGAH RISE,DBS TELOK BLANGAH RISE,90036,26743.489737,28343.943518,1.272607,103.822028,103.822028,90036
4,0,DBS CLEMENTI WEST STREET 2,726,CLEMENTI WEST STREET 2,DBS CLEMENTI WEST STREET 2,120726,20314.909390,31791.216699,1.303782,103.764264,103.764264,120726
6,0,BLK 210 LORONG 8 TOA PAYOH MARKET & FOOD CENTRE,210,LORONG 8 TOA PAYOH,BLK 210 LORONG 8 TOA PAYOH MARKET & FOOD CENTRE,310210,30398.726757,35819.803789,1.340216,103.854873,103.854873,310210
9,0,COMMONWEALTH CRESCENT MARKET,31,COMMONWEALTH CRESCENT,COMMONWEALTH CRESCENT MARKET,149644,24332.824777,32135.918667,1.306900,103.800367,103.800367,149644
...,...,...,...,...,...,...,...,...,...,...,...,...
167,0,CHOMP CHOMP FOOD CENTRE,20,KENSINGTON PARK ROAD,CHOMP CHOMP FOOD CENTRE,557269,31695.705834,38475.442464,1.364233,103.866527,103.866527,557269
168,0,MARKET & HAWKER CENTRE (BLK 448 CLEMENTI AVENU...,448,CLEMENTI AVENUE 3,MARKET & HAWKER CENTRE (BLK 448 CLEMENTI AVENU...,120448,20343.061339,32845.842058,1.313320,103.764517,103.764517,120448
169,0,MARKET & HAWKER CENTRE (BLK 216 BEDOK NORTH ST...,216,BEDOK NORTH STREET 1,MARKET & HAWKER CENTRE (BLK 216 BEDOK NORTH ST...,460216,39117.792247,34365.596824,1.327063,103.933218,103.933218,460216
170,0,DBS EAST COAST LAGOON FOOD VILLAGE,1220,EAST COAST PARKWAY,DBS EAST COAST LAGOON FOOD VILLAGE,468960,39331.556190,32136.743773,1.306906,103.935138,103.935138,468960


In [21]:
hawker_location = set(hawkerlocation_df['address'])

In [22]:
hawker_location

{50032,
 50335,
 51531,
 69111,
 69184,
 81006,
 90011,
 90036,
 100079,
 100082,
 118543,
 120353,
 120448,
 120502,
 120503,
 120726,
 140159,
 141003,
 148813,
 149644,
 150006,
 150079,
 150085,
 150120,
 150163,
 151115,
 160112,
 161001,
 162022,
 168898,
 169982,
 180270,
 198783,
 199583,
 208877,
 210665,
 211041,
 229495,
 247792,
 260007,
 270020,
 270044,
 277700,
 289876,
 310022,
 310074,
 310075,
 310093,
 310127,
 310210,
 320090,
 320092,
 330029,
 330069,
 370080,
 370089,
 371079,
 380017,
 380049,
 380117,
 390051,
 397972,
 402001,
 402004,
 424768,
 430014,
 432004,
 440084,
 441050,
 460016,
 460085,
 460216,
 460511,
 460538,
 461058,
 462208,
 468960,
 469572,
 470630,
 500003,
 521137,
 530105,
 530209,
 555945,
 557269,
 560162,
 560341,
 560409,
 560527,
 560628,
 560724,
 561453,
 564226,
 568226,
 570320,
 574419,
 588215,
 600254,
 600347,
 601037,
 618499,
 640505,
 642221,
 730021,
 760105}

In [23]:
df = pd.DataFrame(hawker_address,columns=['address'])
df = df.astype('int64')

In [24]:
before = set(df['address'])


In [25]:
after = set(hawker_location)


In [26]:
missing = before.difference(after)
list(missing)

[731004, 48940]

Interesting error here, postal code 731004 and 48940 exists in some google searches, but there is a difference in the postal code found in onemap
before : after
731004 : 738937
48940  : 048418

In [27]:
add = "738937"
url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+add
response = requests.get(url)
data = json.loads(response.text)

temp_df = pd.DataFrame.from_dict(data["results"])
temp_df['address'] = 738937
temp_df

,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,20 WOODLANDS CENTRE ROAD SINGAPORE 738937,20,WOODLANDS CENTRE ROAD,NIL,20 WOODLANDS CENTRE ROAD SINGAPORE 738937,738937,20931.3431511633,46747.1611687206,1.43903831450662,103.769799867546,103.769799867546,738937


In [28]:
add = "048418"
url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+add
response = requests.get(url)
data = json.loads(response.text) 

temp_df2 = pd.DataFrame.from_dict(data["results"])
temp_df2['address'] = 48418
temp_df2

,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,MARKET STREET INTERIM HAWKER CENTRE,5,CROSS STREET,MARKET STREET INTERIM HAWKER CENTRE,5 CROSS STREET MARKET STREET INTERIM HAWKER CE...,048418,29731.6582916304,29337.9027270805,1.28159602980757,103.848878196508,103.848878196508,48418


In [29]:
hawkerlocation_df = pd.concat([hawkerlocation_df,temp_df,temp_df2])
hawkerlocation_df = hawkerlocation_df[['LATITUDE','LONGITUDE','address']]
hawkerlocation_df

,LATITUDE,LONGITUDE,address
0,1.443832,103.77721,730021
2,1.272607,103.822028,90036
4,1.303782,103.764264,120726
6,1.340216,103.854873,310210
9,1.3069,103.800367,149644
...,...,...,...
169,1.327063,103.933218,460216
170,1.306906,103.935138,468960
172,1.380988,103.840628,560628
0,1.43903831450662,103.769799867546,738937


In [30]:
mrtlocation_df = pd.read_csv("sgmrt.csv")
mrtlocation_df = mrtlocation_df[['Latitude','Longitude','STN_NAME']]
mrtlocation_df.head()

,Latitude,Longitude,STN_NAME
0,1.440585,103.800998,ADMIRALTY MRT STATION
1,1.316433,103.882893,ALJUNIED MRT STATION
2,1.369933,103.849553,ANG MO KIO MRT STATION
3,1.388093,103.905418,BAKAU LRT STATION
4,1.380018,103.772667,BANGKIT LRT STATION


In [31]:
schinput_df = pd.read_csv('School Location.csv')

#find_postal(schinput_df['school_name'], 'school_coordinates')


In [32]:
schinput_df

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
342,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
343,ZHENGHUA SECONDARY SCHOOL,http://www.zhenghuasec.moe.edu.sg,91 SENJA ROAD,677741,67639455,na,67633577,na,ZHENGHUA_SS@MOE.EDU.SG,JELAPANG LRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
344,ZHONGHUA PRIMARY SCHOOL,http://www.zhonghuapri.moe.edu.sg,12 SERANGOON AVENUE 4,556095,62835413,na,62823882,na,zhonghua_ps@moe.edu.sg,"Ang Mo Kio MRT, Bishan MRT, Serangoon MRT",...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil


In [33]:
schlocation_df = pd.read_csv('school_coordinates.csv')
schlocation_df = schlocation_df.loc[schlocation_df['Unnamed: 0'] == 0]
schlocation_df

,Unnamed: 0,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,0,ADMIRALTY PRIMARY SCHOOL,11,WOODLANDS CIRCLE,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE ADMIRALTY PRIMARY SCHOOL S...,738907,24296.625086,47144.770059,1.442635,103.800040,103.800040,ADMIRALTY PRIMARY SCHOOL
1,0,ADMIRALTY SECONDARY SCHOOL,31,WOODLANDS CRESCENT,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT ADMIRALTY SECONDARY SCHO...,737916,24559.047294,47504.787355,1.445891,103.802398,103.802398,ADMIRALTY SECONDARY SCHOOL
2,0,AHMAD IBRAHIM PRIMARY SCHOOL,10,YISHUN STREET 11,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11 AHMAD IBRAHIM PRIMARY SCHO...,768643,27958.137143,46096.262768,1.433153,103.832942,103.832942,AHMAD IBRAHIM PRIMARY SCHOOL
3,0,AHMAD IBRAHIM SECONDARY SCHOOL,751,YISHUN AVENUE 7,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7 AHMAD IBRAHIM SECONDARY SC...,768928,27599.387289,46417.708755,1.436060,103.829719,103.829719,AHMAD IBRAHIM SECONDARY SCHOOL
4,0,AI TONG SCHOOL,100,BRIGHT HILL DRIVE,AI TONG SCHOOL,100 BRIGHT HILL DRIVE AI TONG SCHOOL SINGAPORE...,579646,27966.808830,38071.919118,1.360583,103.833020,103.833020,AI TONG SCHOOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,0,ZHANGDE PRIMARY SCHOOL,51,JALAN MEMBINA,ZHANGDE PRIMARY SCHOOL,51 JALAN MEMBINA ZHANGDE PRIMARY SCHOOL SINGAP...,169485,27180.146899,29627.107530,1.284212,103.825952,103.825952,ZHANGDE PRIMARY SCHOOL
385,0,ZHENGHUA PRIMARY SCHOOL,9,FAJAR ROAD,ZHENGHUA PRIMARY SCHOOL,9 FAJAR ROAD ZHENGHUA PRIMARY SCHOOL SINGAPORE...,679002,20877.039543,40169.116913,1.379549,103.769314,103.769314,ZHENGHUA PRIMARY SCHOOL
386,0,ZHENGHUA SECONDARY SCHOOL,91,SENJA ROAD,ZHENGHUA SECONDARY SCHOOL,91 SENJA ROAD ZHENGHUA SECONDARY SCHOOL SINGAP...,677741,20453.853994,41144.063893,1.388366,103.765511,103.765511,ZHENGHUA SECONDARY SCHOOL
387,0,ZHONGHUA PRIMARY SCHOOL,12,SERANGOON AVENUE 4,ZHONGHUA PRIMARY SCHOOL,12 SERANGOON AVENUE 4 ZHONGHUA PRIMARY SCHOOL ...,556095,32055.776613,38029.916116,1.360203,103.869762,103.869762,ZHONGHUA PRIMARY SCHOOL


In [34]:
before = set(schinput_df['school_name'])
after = set(schlocation_df['address'])
missing = before.difference(after)
list(missing)

['SCHOOL OF THE ARTS, SINGAPORE']

For this case, its just a difference in the naming 

before                          : after
school of the arts, singapore   : SCHOOL OF THE ARTS (SOTA)

In [35]:
add = "SCHOOL OF THE ARTS"
url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+add
response = requests.get(url)
data = json.loads(response.text) 

temp_df3 = pd.DataFrame.from_dict(data["results"])
temp_df3['address'] = "SCHOOL OF THE ARTS"
temp_df3


,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,SCHOOL OF THE ARTS (SOTA),1,ZUBIR SAID DRIVE,SCHOOL OF THE ARTS (SOTA),1 ZUBIR SAID DRIVE SCHOOL OF THE ARTS (SOTA) S...,227968,29710.4144953834,31302.6820281129,1.29936480411316,103.848687419363,103.848687419363,SCHOOL OF THE ARTS


In [36]:
schlocation_df = pd.concat([schlocation_df,temp_df3])
schlocation_df = schlocation_df[['LATITUDE','LONGITUDE','address']]
schlocation_df

,LATITUDE,LONGITUDE,address
0,1.442635,103.80004,ADMIRALTY PRIMARY SCHOOL
1,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL
2,1.433153,103.832942,AHMAD IBRAHIM PRIMARY SCHOOL
3,1.43606,103.829719,AHMAD IBRAHIM SECONDARY SCHOOL
4,1.360583,103.83302,AI TONG SCHOOL
...,...,...,...
385,1.379549,103.769314,ZHENGHUA PRIMARY SCHOOL
386,1.388366,103.765511,ZHENGHUA SECONDARY SCHOOL
387,1.360203,103.869762,ZHONGHUA PRIMARY SCHOOL
388,1.348407,103.86943,ZHONGHUA SECONDARY SCHOOL


In [37]:
add = "city hall mrt"
url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+add
response = requests.get(url)
data = json.loads(response.text) 

temp_df4 = pd.DataFrame.from_dict(data["results"])

temp_df4['address'] = "city hall mrt"
temp_df4


,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,CITY HALL MRT STATION (EW13 / NS25),150,NORTH BRIDGE ROAD,CITY HALL MRT STATION (EW13 / NS25),150 NORTH BRIDGE ROAD CITY HALL MRT STATION (E...,179100,30144.2503791479,30591.9008128252,1.29293672227779,103.852585580366,103.852585580366,city hall mrt
1,CITY HALL MRT STATION EXIT A,,NIL,CITY HALL MRT STATION EXIT A,CITY HALL MRT STATION EXIT A,NIL,30192.7711374983,30624.6704453975,1.2932330759348,103.853021562271,103.853021562271,city hall mrt
2,CITY HALL MRT STATION EXIT B,,NIL,CITY HALL MRT STATION EXIT B,CITY HALL MRT STATION EXIT B,NIL,30073.6413478041,30610.8982403949,1.29310853308826,103.851951129481,103.851951129481,city hall mrt
3,CITY HALL MRT STATION EXIT C,,NIL,CITY HALL MRT STATION EXIT C,CITY HALL MRT STATION EXIT C,NIL,30188.7357796412,30604.089306118,1.29304694763879,103.852985301431,103.852985301431,city hall mrt
4,CITY HALL MRT STATION EXIT D,,NIL,CITY HALL MRT STATION EXIT D,CITY HALL MRT STATION EXIT D,NIL,30064.1163160084,30654.5545705445,1.29350334620272,103.851865545882,103.851865545882,city hall mrt
5,OCBC CITY HALL MRT STATION,150,NORTH BRIDGE ROAD,OCBC CITY HALL MRT STATION,150 NORTH BRIDGE ROAD OCBC CITY HALL MRT STATI...,179100,30125.128883115,30619.5460454859,1.29318673733671,103.85241376747,103.85241376747,city hall mrt
6,CITY HALL MRT STATION,150,NORTH BRIDGE ROAD,CITY HALL MRT STATION,150 NORTH BRIDGE ROAD CITY HALL MRT STATION SI...,179100,30139.4703683555,30597.7816701867,1.29298990700923,103.852542630339,103.852542630339,city hall mrt
7,UOB CITY HALL MRT STATION,150,NORTH BRIDGE ROAD,UOB CITY HALL MRT STATION,150 NORTH BRIDGE ROAD UOB CITY HALL MRT STATIO...,179100,30139.4703986235,30597.7817004547,1.29298990728295,103.852542630611,103.852542630611,city hall mrt


In [38]:
cbdlocation_df = temp_df4.head(1)

cbdlocation_df = cbdlocation_df[['LATITUDE','LONGITUDE','address']]
cbdlocation_df

,LATITUDE,LONGITUDE,address
0,1.29293672227779,103.852585580366,city hall mrt


In [39]:
#Summarised location dataframes

In [40]:
cols = cbdlocation_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
cbdlocation_df = cbdlocation_df[cols]
cbdlocation_df.head()


,address,LATITUDE,LONGITUDE
0,city hall mrt,1.29293672227779,103.852585580366


In [41]:
cols = hdblocation_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
hdblocation_df = hdblocation_df[cols]
hdblocation_df.head()


,address,LATITUDE,LONGITUDE
0,215 LOR 8 TOA PAYOH,1.341828,103.855108
1,118 LOR 1 TOA PAYOH,1.339563,103.844894
2,699 HOUGANG ST 52,1.375921,103.889190
3,483 CHOA CHU KANG AVE 5,1.375841,103.737757
5,486 ADMIRALTY LINK,1.456027,103.816100


In [42]:
cols = hawkerlocation_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
hawkerlocation_df = hawkerlocation_df[cols]
hawkerlocation_df.head()



,address,LATITUDE,LONGITUDE
0,730021,1.443832,103.77721
2,90036,1.272607,103.822028
4,120726,1.303782,103.764264
6,310210,1.340216,103.854873
9,149644,1.3069,103.800367


In [43]:
cols = schlocation_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
schlocation_df = schlocation_df[cols]
schlocation_df.head()


,address,LATITUDE,LONGITUDE
0,ADMIRALTY PRIMARY SCHOOL,1.442635,103.80004
1,ADMIRALTY SECONDARY SCHOOL,1.445891,103.802398
2,AHMAD IBRAHIM PRIMARY SCHOOL,1.433153,103.832942
3,AHMAD IBRAHIM SECONDARY SCHOOL,1.43606,103.829719
4,AI TONG SCHOOL,1.360583,103.83302


In [44]:
cols = mrtlocation_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
mrtlocation_df = mrtlocation_df[cols]
mrtlocation_df.head()

,STN_NAME,Latitude,Longitude
0,ADMIRALTY MRT STATION,1.440585,103.800998
1,ALJUNIED MRT STATION,1.316433,103.882893
2,ANG MO KIO MRT STATION,1.369933,103.849553
3,BAKAU LRT STATION,1.388093,103.905418
4,BANGKIT LRT STATION,1.380018,103.772667


In [45]:
from geopy.distance import geodesic

def find_nearest(house, amenity, results):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    """
    
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)

            if distance < flat_amenity[2]:
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [46]:
# mrtdist = {}
# find_nearest(hdblocation_df, mrtlocation_df,mrtdist)
# mrtdist

In [47]:
# hawkerdist = {}
# find_nearest(hdblocation_df, hawkerlocation_df,hawkerdist)
# hawkerdist

In [48]:
# cbddist = {}
# find_nearest(hdblocation_df, cbdlocation_df,cbddist)
# cbddist

In [49]:
# schdist = {}
# find_nearest(hdblocation_df, schlocation_df,schdist)
# schdist

In [50]:
# mrtdist_df = pd.DataFrame.from_dict(mrtdist, orient='index',
#                        columns=['address', 'name','distance_mrt'])
# mrtdist_df.head()                       

In [51]:
# cbddist_df = pd.DataFrame.from_dict(cbddist, orient='index',
#                        columns=['address', 'name','distance_cbd'])
# cbddist_df.head()       

In [52]:
# hawkerdist_df = pd.DataFrame.from_dict(hawkerdist, orient='index',
#                        columns=['address','name', 'distance_hawker'])
# hawkerdist_df.head()       

In [53]:
# schdist_df = pd.DataFrame.from_dict(schdist, orient='index',
#                        columns=['address','name', 'distance_sch'])
# schdist_df.head()       

In [54]:
# temp_df = schdist_df.merge(hawkerdist_df, left_on='address', right_on='address')



In [55]:
# temp_df = temp_df.merge(cbddist_df, left_on='address', right_on='address')
# temp_df = temp_df.merge(mrtdist_df, left_on='address', right_on='address')

In [56]:
# temp_df

In [57]:
# temp_df.to_csv('all_amenities_dist.csv')

In [58]:
all_amenities_dist_df = pd.read_csv('all_amenities_dist.csv')

In [59]:
value_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,108 ANG MO KIO AVE 4
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,602 ANG MO KIO AVE 5
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,465 ANG MO KIO AVE 10
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,601 ANG MO KIO AVE 5
...,...,...,...,...,...,...,...,...,...,...,...,...
122671,2022-03,YISHUN,EXECUTIVE,355,YISHUN RING RD,10 TO 12,146.0,Maisonette,1988,65 years 08 months,800000.0,355 YISHUN RING RD
122672,2022-03,YISHUN,EXECUTIVE,277,YISHUN ST 22,10 TO 12,146.0,Maisonette,1985,62 years 06 months,788000.0,277 YISHUN ST 22
122673,2022-03,YISHUN,EXECUTIVE,824,YISHUN ST 81,01 TO 03,142.0,Apartment,1987,64 years 09 months,800000.0,824 YISHUN ST 81
122674,2022-03,YISHUN,EXECUTIVE,828,YISHUN ST 81,01 TO 03,142.0,Apartment,1988,64 years 11 months,723000.0,828 YISHUN ST 81


In [60]:
complied_df = value_df.merge(all_amenities_dist_df, left_on='address', right_on='address')

In [61]:
complied_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,address,Unnamed: 0,name_x,distance_sch,name_y,distance_hawker,name_x.1,distance_cbd,name_y.1,distance_mrt
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,406 ANG MO KIO AVE 10,3798,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,city hall mrt,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 01 month,...,406 ANG MO KIO AVE 10,3798,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,city hall mrt,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,...,406 ANG MO KIO AVE 10,3798,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,city hall mrt,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,...,406 ANG MO KIO AVE 10,3798,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,city hall mrt,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,60 years 01 month,...,406 ANG MO KIO AVE 10,3798,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,city hall mrt,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122671,2022-03,YISHUN,3 ROOM,813B,YISHUN RING RD,07 TO 09,67.0,Model A,2018,95 years 03 months,...,813B YISHUN RING RD,314,PEIYING PRIMARY SCHOOL,0.2178494626658036 km,760105,1.6536747255974675 km,city hall mrt,13.944026951573466 km,KHATIB MRT STATION,0.06805026518778176 km
122672,2022-03,YISHUN,3 ROOM,813B,YISHUN RING RD,07 TO 09,67.0,Model A,2018,95 years 03 months,...,813B YISHUN RING RD,314,PEIYING PRIMARY SCHOOL,0.2178494626658036 km,760105,1.6536747255974675 km,city hall mrt,13.944026951573466 km,KHATIB MRT STATION,0.06805026518778176 km
122673,2022-03,YISHUN,4 ROOM,672C,YISHUN AVE 4,10 TO 12,93.0,Model A,2018,94 years 11 months,...,672C YISHUN AVE 4,1607,NORTHLAND PRIMARY SCHOOL,0.3463494722480279 km,760105,2.190533772663818 km,city hall mrt,13.912771884353369 km,KHATIB MRT STATION,1.0816216255248843 km
122674,2022-03,YISHUN,4 ROOM,509B,YISHUN AVE 4,13 TO 15,93.0,Model A,2018,95 years 02 months,...,509B YISHUN AVE 4,1283,ORCHID PARK SECONDARY SCHOOL,0.30370391055582474 km,760105,2.390471435640472 km,city hall mrt,13.436090410345793 km,KHATIB MRT STATION,0.8890922092840777 km


In [62]:
complied_df = complied_df.drop(columns=['Unnamed: 0','name_x.1'])
complied_df = complied_df.rename(columns={
                                        'name_x': 'nearest_school_name',
                                        'name_y': 'nearest_hawker_postalcode',
                                        'name_y.1': 'nearest_mrt_name'
                                        }
                                        )

In [63]:
complied_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,nearest_school_name,distance_sch,nearest_hawker_postalcode,distance_hawker,distance_cbd,nearest_mrt_name,distance_mrt
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 01 month,235000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,202000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,210000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,60 years 01 month,220000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026 km,560409,0.18187892682440665 km,7.63852334091251 km,ANG MO KIO MRT STATION,1.0002190239027458 km


In [64]:
complied_df.dtypes

month                         object
town                          object
flat_type                     object
block                         object
street_name                   object
storey_range                  object
floor_area_sqm               float64
flat_model                    object
lease_commence_date            int64
remaining_lease               object
resale_price                 float64
address                       object
nearest_school_name           object
distance_sch                  object
nearest_hawker_postalcode      int64
distance_hawker               object
distance_cbd                  object
nearest_mrt_name              object
distance_mrt                  object
dtype: object

In [65]:
complied_df['month'].astype('datetime64')

0        2017-01-01
1        2017-05-01
2        2018-03-01
3        2018-03-01
4        2018-05-01
            ...    
122671   2022-03-01
122672   2022-03-01
122673   2022-03-01
122674   2022-03-01
122675   2022-03-01
Name: month, Length: 122676, dtype: datetime64[ns]

In [66]:
complied_df["distance_sch"] = complied_df["distance_sch"].str[:-3]


In [67]:
complied_df["distance_hawker"] = complied_df["distance_hawker"].str[:-3]
complied_df["distance_cbd"] = complied_df["distance_cbd"].str[:-3]
complied_df["distance_mrt"] = complied_df["distance_mrt"].str[:-3]

In [68]:
complied_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,nearest_school_name,distance_sch,nearest_hawker_postalcode,distance_hawker,distance_cbd,nearest_mrt_name,distance_mrt
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026,560409,0.18187892682440665,7.63852334091251,ANG MO KIO MRT STATION,1.0002190239027458
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 01 month,235000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026,560409,0.18187892682440665,7.63852334091251,ANG MO KIO MRT STATION,1.0002190239027458
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,202000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026,560409,0.18187892682440665,7.63852334091251,ANG MO KIO MRT STATION,1.0002190239027458
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,210000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026,560409,0.18187892682440665,7.63852334091251,ANG MO KIO MRT STATION,1.0002190239027458
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,60 years 01 month,220000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.2181252401285026,560409,0.18187892682440665,7.63852334091251,ANG MO KIO MRT STATION,1.0002190239027458


In [69]:
complied_df["distance_sch"]  = complied_df["distance_sch"].astype('float')
complied_df["distance_hawker"] = complied_df["distance_hawker"].astype('float')
complied_df["distance_cbd"] = complied_df["distance_cbd"].astype('float')
complied_df["distance_mrt"] = complied_df["distance_mrt"].astype('float')

In [70]:
complied_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,nearest_school_name,distance_sch,nearest_hawker_postalcode,distance_hawker,distance_cbd,nearest_mrt_name,distance_mrt
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.218125,560409,0.181879,7.638523,ANG MO KIO MRT STATION,1.000219
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 01 month,235000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.218125,560409,0.181879,7.638523,ANG MO KIO MRT STATION,1.000219
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,202000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.218125,560409,0.181879,7.638523,ANG MO KIO MRT STATION,1.000219
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,210000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.218125,560409,0.181879,7.638523,ANG MO KIO MRT STATION,1.000219
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,60 years 01 month,220000.0,406 ANG MO KIO AVE 10,TOWNSVILLE PRIMARY SCHOOL,0.218125,560409,0.181879,7.638523,ANG MO KIO MRT STATION,1.000219


In [71]:
complied_df.dtypes

month                         object
town                          object
flat_type                     object
block                         object
street_name                   object
storey_range                  object
floor_area_sqm               float64
flat_model                    object
lease_commence_date            int64
remaining_lease               object
resale_price                 float64
address                       object
nearest_school_name           object
distance_sch                 float64
nearest_hawker_postalcode      int64
distance_hawker              float64
distance_cbd                 float64
nearest_mrt_name              object
distance_mrt                 float64
dtype: object

In [72]:
hdblocation_df

,address,LATITUDE,LONGITUDE
0,215 LOR 8 TOA PAYOH,1.341828,103.855108
1,118 LOR 1 TOA PAYOH,1.339563,103.844894
2,699 HOUGANG ST 52,1.375921,103.889190
3,483 CHOA CHU KANG AVE 5,1.375841,103.737757
5,486 ADMIRALTY LINK,1.456027,103.816100
...,...,...,...
11383,224A COMPASSVALE WALK,1.387882,103.898019
11384,212 JURONG EAST ST 21,1.339094,103.740392
11385,681C WOODLANDS DR 62,1.439789,103.803120
11386,403B FERNVALE LANE,1.388791,103.872980


In [73]:
complied_df = complied_df.merge(hdblocation_df,left_on = 'address', right_on='address')

In [74]:
complied_df.to_excel('resalevalue_withdist.xlsx')